Version 1.0.0

# Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise.

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.

Now go through the notebook and complete the assignment.

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse

# Load the data

In [ ]:
!wget "https://raw.githubusercontent.com/hse-aml/competitive-data-science/refs/heads/master/readonly/data_leakages_data/test_pairs.csv"

--2025-11-20 12:50:48--  https://raw.githubusercontent.com/hse-aml/competitive-data-science/refs/heads/master/readonly/data_leakages_data/test_pairs.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6491404 (6.2M) [text/plain]
Saving to: ‘test_pairs.csv’

test_pairs.csv      100%[===================>]   6.19M  --.-KB/s    in 0.08s   

2025-11-20 12:50:49 (76.3 MB/s) - ‘test_pairs.csv’ saved [6491404/6491404]



Let's load the test data. Note, that we don't have any training data here, just test data. Moreover, *we will not even use any features* of test objects. All we need to solve this task is the file with the indices for the pairs, that we need to compare.

Let's load the data with test indices.

In [ ]:
test = pd.read_csv('test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


For example, we can think that there is a test dataset of images, and each image is assigned a unique `Id` from $0$ to $N-1$ (N -- is the number of images). In the dataframe from above `FirstId` and `SecondId` point to these `Id`'s and define pairs, that we should compare: e.g. do both images in the pair belong to the same class or not. So, for example for the first row: if images with `Id=1427` and `Id=8053` belong to the same class, we should predict $1$, and $0$ otherwise.

But in our case we don't really care about the images, and how exactly we compare the images (as long as comparator is binary).  

**We suggest you to try to solve the puzzle yourself first.** You need to submit a `.csv` file with columns `pairId` and `Prediction` to the grader. The number of submissions allowed is made pretty huge to let you explore the data without worries. The returned score should be very close to $1$.

**If you do not want to think much** -- scroll down and follow the instructions below.

In [ ]:
test.describe()

,pairId,FirstId,SecondId
count,368550.000000,368550.000000,368550.000000
mean,184274.500000,10863.601118,11950.398882
std,106391.365192,7280.190939,7602.814820
min,0.000000,0.000000,0.000000
25%,92137.250000,4574.000000,5698.000000
50%,184274.500000,9886.000000,10512.000000
75%,276411.750000,16512.000000,18782.000000
max,368549.000000,26324.000000,26324.000000


# EDA and leakage intuition

As we already know, the key to discover data leakages is careful EDA. So let's start our work with some basic data exploration and build an intuition about the leakage.

First, check, how many different `id`s are there: concatenate `FirstId` and `SecondId` and print the number of unique elements. Also print minimum and maximum value for that vector.

In [ ]:
all_unique_ids = pd.concat([test['FirstId'], test['SecondId']], ignore_index=True) # concat on the first on top of the second
all_unique_ids.head()

,0
0,1427
1,17044
2,19237
3,8005
4,16837


In [ ]:
# count number of unique
uniq = all_unique_ids.nunique()
uniq

26325

In [ ]:
max_id, min_id = all_unique_ids.max(), all_unique_ids.min()
max_id, min_id

(26324, 0)

and then print how many pairs we need to classify (it is basically the number of rows in the test set)

In [ ]:
len(test)

368550

Now print, how many distinct pairs it would be possible to create out of all "images" in the dataset?   

In [ ]:
# here total rows in the orignal df is around 368k while we have only 26k unique ids
# that means from these 26k unique ideas the combinations for each pair has been made
# therefore number of combinatons to fill for 2 positions: (N * (N-1)) // 2 for unordered arranngement. (a,b) == (b,a)

total_combs = (uniq * (uniq-1)) // 2
print(f"{total_combs:,}")

346,489,650


So the number of pairs we are given to classify is very very small compared to the total number of pairs.

To exploit the leak we need to **assume (or prove)**, that the total number of positive pairs is small, compared to the total number of pairs. For example: think about an image dataset with $1000$ classes, $N$ images per class. Then if the task was to tell whether a pair of images belongs to the same class or not, we would have $1000\frac{N(N-1)}{2}$ positive pairs, while total number of pairs was $\frac{1000N(1000N - 1)}{2}$.

Another example: in [Quora competitition](https://www.kaggle.com/c/quora-question-pairs) the task was to classify whether a pair of qustions are duplicates of each other or not. Of course, total number of question pairs is very huge, while number of duplicates (positive pairs) is much much smaller.


**Notes:**
Thinking N as 100, for every class where all images are similar to one another, number of poitive pairs for one class = N choose 2. Hence, if 1000 classes it would be the above formula!

Finally, let's get a fraction of pairs of class `1`. We just need to submit a constant prediction "all ones" and check the returned accuracy. Create a dataframe with columns `pairId` and `Prediction`, fill it and export it to `.csv` file. Then submit to grader and examine grader's output.

In [ ]:
# I couldn't submit to grader since the course is delisted!

So, we assumed the total number of pairs is much higher than the number of positive pairs, but it is not the case for the test set. It means that the test set is constructed not by sampling random pairs, but with a specific sampling algorithm. Pairs of class `1` are oversampled.

Now think, how we can exploit this fact? What is the leak here? If you get it now, you may try to get to the final answer yourself, othewise you can follow the instructions below.   

# Building a magic feature

In this section we will build a magic feature, that will solve the problem almost perfectly. The instructions will lead you to the correct solution, but please, try to explain the purpose of the steps we do to yourself -- it is very important.

## Incidence matrix

First, we need to build an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix). You can think of pairs `(FirstId, SecondId)` as of edges in an undirected graph.

The incidence matrix is a matrix of size `(maxId + 1, maxId + 1)`, where each row (column) `i` corresponds `i-th` `Id`. In this matrix we put the value `1` to the position `[i, j]`, if and only if a pair `(i, j)` or `(j, i)` is present in  a given set of pais `(FirstId, SecondId)`. All the other elements in the incidence matrix are zeros.   

**Important!** The incidence matrices are typically very very sparse (small number of non-zero values). At the same time incidence matrices are usually huge in terms of total number of elements, and it is **impossible to store them in memory in dense format**. But due to their sparsity incidence matrices **can be easily represented as sparse matrices**. If you are not familiar with sparse matrices, please see [wiki](https://en.wikipedia.org/wiki/Sparse_matrix) and [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html). Please, use any of `scipy.sparse` constructors to build incidence matrix.

For example, you can use this constructor: `scipy.sparse.coo_matrix((data, (i, j)))`. We highly recommend to learn to use different `scipy.sparse` constuctors, and matrices types, but if you feel you don't want to use them, you can always build this matrix with a simple `for` loop. You will need first to create a matrix using `scipy.sparse.coo_matrix((M, N), [dtype])` with an appropriate shape `(M, N)` and then iterate through `(FirstId, SecondId)` pairs and fill corresponding elements in matrix with ones.

**Note**, that the matrix should be symmetric and consist only of zeros and ones. It is a way to check yourself.

In [ ]:
"""
Example:

FirstId (Row)   SecondId (Col)
0	            1
1	            2
0	            1

The final lists (after concatenation) look like this:
row = [0, 1, 0, 1, 2, 1]
col = [1, 2, 1, 0, 1, 0]
data = [1, 1, 1, 1, 1, 1] (all ones)
"""

'\nExample:\nThe final lists (after concatenation) look like this:\nrow = [0, 1, 0, 1, 2, 1]\ncol = [1, 2, 1, 0, 1, 0]\ndata = [1, 1, 1, 1, 1, 1] (all ones) \n'

In [ ]:
N = max_id+1
# row -> col edges and data is what to be put at the matrix[row][col] = data
row = np.concat([test['FirstId'].to_numpy(), test['SecondId'].to_numpy()])
col = np.concat([test['SecondId'].to_numpy(), test['FirstId'].to_numpy()])
data = np.ones(len(row), dtype=int)

# sparse matrix with (rows, cols) to place the data, but there maybe duplicates so -> csr, so that duplicates are summed at every location
inc_mat = scipy.sparse.coo_matrix((data, (row, col)), shape=(N, N)).tocsr()

# clip all entries in the matrix where  there is data to 1
inc_mat.data[:] = 1

# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872

It is convenient to have matrix in `csr` format eventually.
When a scipy.sparse.coo_matrix is converted to other formats like csr or csc, duplicate (row, col) entries are automatically summed together.

## Now build the magic feature

Why did we build the incidence matrix? We can think of the rows in this matix as of representations for the objects. `i-th` row is a representation for an object with `Id = i`. Then, to measure similarity between two objects we can measure similarity between their representations. And we will see, that such representations are very good.

Now select the rows from the incidence matrix, that correspond to `test.FirstId`'s, and `test.SecondId`'s.

In [ ]:
# Note, scipy goes crazy if a matrix is indexed with pandas' series.
# So do not forget to convert `pd.series` to `np.array`
# These lines should normally run very quickly

rows_FirstId   = inc_mat[test.FirstId]
rows_SecondId  = inc_mat[test.SecondId]

print(f"FirstId shape: {rows_FirstId.shape}")
print(f"SecondId shape: {rows_SecondId.shape}")

FirstId shape: (368550, 26325)
SecondId shape: (368550, 26325)


Our magic feature will be the *dot product* between representations of a pair of objects. Dot product can be regarded as similarity measure -- for our non-negative representations the dot product is close to 0 when the representations are different, and is huge, when representations are similar.

Now compute dot product between corresponding rows in `rows_FirstId` and `rows_SecondId` matrices.

In [ ]:
# special way to multply scipy matrices
(rows_FirstId.multiply(rows_SecondId)).shape

(368550, 26325)

In [ ]:
# Note, that in order to do pointwise multiplication in scipy.sparse you need to use function `multiply`
# regular `*` corresponds to matrix-matrix multiplication

# sum along rows, A1 flattens from (x, 1) to (x, ) esp for scipy matrices
f = rows_FirstId.multiply(rows_SecondId).sum(axis=1).A1

# Sanity check
assert f.shape == (368550, )

In [ ]:
f[:10] # scores of similarity for each row(pair)

array([20, 14, 20, 20, 14, 20, 14, 14, 14, 14])

That is it! **We've built our magic feature.**

# From magic feature to binary predictions

But how do we convert this feature into binary predictions? We do not have a train set to learn a model, but we have a piece of information about test set: the baseline accuracy score that you got, when submitting constant. And we also have a very strong considerations about the data generative process, so probably we will be fine even without a training set.

We may try to choose a thresold, and set the predictions to 1, if the feature value `f` is higer than the threshold, and 0 otherwise. What threshold would you choose?

How do we find a right threshold? Let's first examine this feature: print frequencies (or counts) of each value in the feature `f`.

In [ ]:
type(f)

numpy.ndarray

In [ ]:
pd.Series(f).value_counts()

,count
20,183799
14,183279
15,852
19,546
28,54
35,14
21,6


Do you see how this feature clusters the pairs? Maybe you can guess a good threshold by looking at the values?

In fact, in other situations it can be not that obvious, but in general to pick a threshold you only need to remember the score of your baseline submission and use this information. Do you understand why and how?  

In [ ]:
len(test) * .90

331695.0

In [ ]:
"""
So here is the idea, suppose the number of ones are the accuracy we got when submitted is 90% then we just want to know how many examples constitutes 84% of total data.
Once we get the number, then we can just decide the value of f which clearly demarcates this.
For our assumption:

90% of 368550 = 331,695
So choosing value of 'f' that puts around 309,582 on one side and the rest on the other side!
There aint a clear f which perfectly seperates 331695 on one side and the rest on other, our best option is f==14 or f==20, we pred '1'.

"""

"\nSo here is the idea, suppose the number of ones are the accuracy we got when submitted is 90% then we just want to know how many examples constitutes 84% of total data.\nOnce we get the number, then we can just decide the value of f which clearly demarcates this.\nFor our assumption:\n\n90% of 368550 = 331,695\nSo choosing value of 'f' that puts around 309,582 on one side and the rest on the other side!\nThere aint a clear f which perfectly seperates 331695 on one side and the rest on other, our best option is f==14 or f==20, we pred '1'.\n\n"

In [ ]:
183799 + 183279

367078

Choose a threshold below:

In [ ]:
pred = f > 14

# Finally, let's create a submission

In [ ]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

submission.to_csv('submission.csv', index=False)

Now submit it to the grader! It is not possible to submit directly from this notebook, as we need to submit a `csv` file, not a single number (limitation of Coursera platform).

To download `submission.csv` file that you've just produced <a href='./submission.csv'>click here</a> (if the link opens in browser, right-click on it and shoose "Save link as"). Then go to [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.


If you did everything right, the score should be very high.

**Finally:** try to explain to yourself, why the whole thing worked out. In fact, there is no magic in this feature, and the idea to use rows in the incidence matrix can be intuitively justified.

# Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!